# Washington State DOT Traffic API

source: [https://wsdot.wa.gov/traffic/api](https://wsdot.wa.gov/traffic/api)

In [58]:
import pandas as pd
import json
import os
import requests

from dotenv import load_dotenv
from pathlib import Path
from sqlalchemy import create_engine

## Import env variables

In [59]:
# load the .env file
load_dotenv()

# API variables
api_key = os.getenv("API_KEY")

# MySQL database variables
db_user = os.getenv("AZURE_USERNAME")
db_pwd = os.getenv("AZURE_PWD")
db_host = os.getenv("AZURE_URL")
db_port = os.getenv("AZURE_PORT")
db_database = os.getenv("AZURE_DB")

# print(api_key)

## URLs to Access APIs

In [32]:
TRAVEL_TIMES_URL = "http://wsdot.wa.gov/Traffic/api/TravelTimes/TravelTimesREST.svc/GetTravelTimesAsJson?AccessCode={ACCESSCODE}"

TRAFFIC_ALERTS_URL = "http://www.wsdot.wa.gov/Traffic/api/HighwayAlerts/HighwayAlertsREST.svc/GetAlertsAsJson?AccessCode={ACCESSCODE}"

WEATHER_INFORMATION_URL = "http://wsdot.wa.gov/Traffic/api/WeatherInformation/WeatherInformationREST.svc/GetCurrentWeatherInformationAsJson?AccessCode={ACCESSCODE}"

### Custom function to get API data

In [60]:
def get_api_data(url, access_key):
    """_summary_

    Args:
        url (_type_): _description_
        access_key (_type_): _description_

    Returns:
        _type_: _description_
    """
    # create the url with the access key
    url_api = url.format(ACCESSCODE=access_key)
    response = requests.get(url_api)

    # check if request was successful
    if response.status_code == 200:
        print("Data fetched successfully.")
        data = response.json()
        df = pd.DataFrame(data)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

    # return the dataframe and the status code
    return df, response.status_code

In [61]:
def convert_dict_to_json(dataframe):

    for col in dataframe.columns:
        if dataframe[col].apply(lambda x: isinstance(x, dict)).all():
            dataframe[col] = dataframe[col].apply(lambda x: json.dumps(x))

In [75]:
def get_updates(
    travel_times_url, traffic_alerts_url, weather_information_url, access_key
):

    # Get current time
    timestamp = pd.Timestamp.now()
    df_tt, response_tt = get_api_data(travel_times_url, access_key)
    df_ta, response_ta = get_api_data(traffic_alerts_url, access_key)
    df_wa, response_wa = get_api_data(weather_information_url, access_key)

    # add pull_date_time
    df_tt["timestamp"] = timestamp
    df_ta["timestamp"] = timestamp
    df_wa["timestamp"] = timestamp

    # assign column to use as foreign key
    df_tt["tt_id"] = 1
    df_ta["ta_id"] = 2
    df_wa["wa_id"] = 3

    # convert dict columns to json
    convert_dict_to_json(df_tt)
    convert_dict_to_json(df_ta)
    convert_dict_to_json(df_wa)

    df_api_fetch = pd.DataFrame(
        [
            {
                "timestamp": timestamp,
                "travel_times_response": response_tt,
                "traffic_alerts_response": response_ta,
                "weather_alerts_response": response_wa,
                "tt_id": 1,
                "ta_id": 2,
                "wa_id": 3,
            }
        ]
    )

    return df_api_fetch, df_tt, df_ta, df_wa

# EXTRACT

## Call to get all info from API

In [76]:
df_api_fetch, df_tt, df_ta, df_wa = get_updates(
    TRAVEL_TIMES_URL, TRAFFIC_ALERTS_URL, WEATHER_INFORMATION_URL, api_key
)

Data fetched successfully.
Data fetched successfully.
Data fetched successfully.


# LOAD 

## Export to CSV

In [77]:
# Set path to datasets folder
datasets = Path("../datasets")
df_api_fetch.to_csv(datasets / "api_fetch.csv", index=False)
df_tt.to_csv(datasets / "travel_times.csv", index=False)
df_ta.to_csv(datasets / "traffic_alerts.csv", index=False)
df_wa.to_csv(datasets / "weather_alerts.csv", index=False)

## Connection to MySQL Database

In [78]:
# Connect to database
connection_url = (
    f"mysql+pymysql://{db_user}:{db_pwd}" f"@{db_host}:{db_port}/{db_database}"
)
try:
    engine = create_engine(connection_url)
    print("Connection successful!")
except Exception as e:
    print("Connection unsuccessful! The error is: ", e)

Connection successful!


In [38]:
display(df_tt)

,AverageTime,CurrentTime,Description,Distance,EndPoint,Name,StartPoint,TimeUpdated,TravelTimeID,timestamp,tt_id
0,25,25,Everett to Downtown Seattle using HOV lanes,26.72,{'Description': 'I-5 @ University St in Seattl...,Everett-Seattle HOV,"{'Description': 'I-5 @ 41st St in Everett', 'D...",/Date(1738647600000-0800)/,2,2025-02-03 21:41:39.776339,1
1,25,25,Downtown Seattle to Everett using HOV lanes,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Seattle-Everett HOV,{'Description': 'I-5 @ University St in Seattl...,/Date(1738647600000-0800)/,3,2025-02-03 21:41:39.776339,1
2,25,25,Downtown Seattle to Everett,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Seattle-Everett,{'Description': 'I-5 @ University St in Seattl...,/Date(1738647600000-0800)/,4,2025-02-03 21:41:39.776339,1
3,9,9,Downtown Bellevue to Issaquah,9.28,"{'Description': 'I-90 @ Front St in Issaquah',...",Bellevue-Issaquah,{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1738647600000-0800)/,5,2025-02-03 21:41:39.776339,1
4,9,9,Downtown Bellevue to Issaquah using HOV lanes,9.28,"{'Description': 'I-90 @ Front St in Issaquah',...",Bellevue-Issaquah HOV,{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1738647600000-0800)/,6,2025-02-03 21:41:39.776339,1
...,...,...,...,...,...,...,...,...,...,...,...
163,8,5,SR500 WB and 112th Ave to SR500 and I-5 JCT,4.80,{'Description': 'SR500 and I-5 JCT in Vancouve...,021 SR500 WB and 112th Ave to SR500 and I-5 JCT,{'Description': 'SR500 WB and 112th Ave in Van...,/Date(1738647600000-0800)/,528,2025-02-03 21:41:39.776339,1
164,19,18,SR500 WB and 112th Ave to I-5 to I-5 at I-84 JCT,14.16,"{'Description': 'I-5 at I-84 JCT in Portland',...",021 SR500 WB and 112th Ave to I-5 to I-5 at I-...,{'Description': 'SR500 WB and 112th Ave in Van...,/Date(1738647600000-0800)/,529,2025-02-03 21:41:39.776339,1
165,20,20,SR500 WB and 112th Ave to I-205 to I-84 to I-5...,17.42,"{'Description': 'I-84 to I-5 JCT in Portland',...",022 SR500 WB and 112th Ave to I-205 to I-84 to...,{'Description': 'SR500 WB and 112th Ave in Van...,/Date(1738647600000-0800)/,530,2025-02-03 21:41:39.776339,1
166,16,17,SB I-5 54th Ave To JBLM Main Gate,17.10,"{'Description': 'I-5 @ JBLM Main Gate', 'Direc...","SB I-5, Fife To JBLM Main Gate","{'Description': 'I-5 @ 54th Ave', 'Direction':...",/Date(1738647600000-0800)/,531,2025-02-03 21:41:39.776339,1


## Upload Data to MySQL

In [54]:
df_tt.head(10)

,AverageTime,CurrentTime,Description,Distance,EndPoint,Name,StartPoint,TimeUpdated,TravelTimeID,timestamp,tt_id
0,25,25,Everett to Downtown Seattle using HOV lanes,26.72,{'Description': 'I-5 @ University St in Seattl...,Everett-Seattle HOV,"{'Description': 'I-5 @ 41st St in Everett', 'D...",/Date(1738647600000-0800)/,2,2025-02-03 21:41:39.776339,1
1,25,25,Downtown Seattle to Everett using HOV lanes,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Seattle-Everett HOV,{'Description': 'I-5 @ University St in Seattl...,/Date(1738647600000-0800)/,3,2025-02-03 21:41:39.776339,1
2,25,25,Downtown Seattle to Everett,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Seattle-Everett,{'Description': 'I-5 @ University St in Seattl...,/Date(1738647600000-0800)/,4,2025-02-03 21:41:39.776339,1
3,9,9,Downtown Bellevue to Issaquah,9.28,"{'Description': 'I-90 @ Front St in Issaquah',...",Bellevue-Issaquah,{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1738647600000-0800)/,5,2025-02-03 21:41:39.776339,1
4,9,9,Downtown Bellevue to Issaquah using HOV lanes,9.28,"{'Description': 'I-90 @ Front St in Issaquah',...",Bellevue-Issaquah HOV,{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1738647600000-0800)/,6,2025-02-03 21:41:39.776339,1
5,9,9,Issaquah to Downtown Bellevue using HOV lanes,9.25,{'Description': 'I-405 @ NE 8th St in Bellevue...,Issaquah-Bellevue HOV,"{'Description': 'I-90 @ Front St in Issaquah',...",/Date(1738647600000-0800)/,7,2025-02-03 21:41:39.776339,1
6,9,9,Issaquah to Downtown Bellevue,9.25,{'Description': 'I-405 @ NE 8th St in Bellevue...,Issaquah-Bellevue,"{'Description': 'I-90 @ Front St in Issaquah',...",/Date(1738647600000-0800)/,8,2025-02-03 21:41:39.776339,1
7,25,25,Downtown Bellevue to Everett,26.76,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Bellevue-Everett,{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1738647600000-0800)/,9,2025-02-03 21:41:39.776339,1
8,25,25,Everett to Downtown Bellevue,26.86,{'Description': 'I-405 @ NE 8th St in Bellevue...,Everett-Bellevue,"{'Description': 'I-5 @ 41st St in Everett', 'D...",/Date(1738647600000-0800)/,10,2025-02-03 21:41:39.776339,1
9,25,25,Downtown Bellevue to Everett using HOV lanes,26.76,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Bellevue-Everett HOV,{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1738647600000-0800)/,11,2025-02-03 21:41:39.776339,1


In [74]:
# Table api_fetch
df_api_fetch.to_sql("api_fetch", con=engine, if_exists="append", index=False)

# Table time_trave_raw
df_tt.to_sql("time_travel_raw", con=engine, if_exists="append", index=False)

# Table traffic_alerts_raw
df_ta.to_sql("traffic_alerts_raw", con=engine, if_exists="append", index=False)

# Table weather_alerts_raw
df_wa.to_sql("weather_alerts_raw", con=engine, if_exists="append", index=False)

print("Data saved to database.")

IntegrityError: (pymysql.err.IntegrityError) (1452, 'Cannot add or update a child row: a foreign key constraint fails (`api_fetch_raw`.`time_travel_raw`, CONSTRAINT `fk_tt_fetch` FOREIGN KEY (`tt_id`) REFERENCES `api_fetch` (`id`))')
[SQL: INSERT INTO time_travel_raw (`AverageTime`, `CurrentTime`, `Description`, `Distance`, `EndPoint`, `Name`, `StartPoint`, `TimeUpdated`, `TravelTimeID`, timestamp, tt_id) VALUES (%(AverageTime)s, %(CurrentTime)s, %(Description)s, %(Distance)s, %(EndPoint)s, %(Name)s, %(StartPoint)s, %(TimeUpdated)s, %(TravelTimeID)s, %(timestamp)s, %(tt_id)s)]
[parameters: [{'AverageTime': 25, 'CurrentTime': 25, 'Description': 'Everett to Downtown Seattle using HOV lanes', 'Distance': 26.72, 'EndPoint': '{"Description": "I-5 @ University St in Seattle", "Direction": "S", "Latitude": 47.609294, "Longitude": -122.331759, "MilePost": 165.83, "RoadName": "005"}', 'Name': 'Everett-Seattle HOV', 'StartPoint': '{"Description": "I-5 @ 41st St in Everett", "Direction": "S", "La

In [28]:
df_response.head()

,timestamp,travel_times_response,traffic_alerts_response,weather_alerts_response,tt_key,ta_key,wa_key
0,2025-02-03 21:40:15.753008,200,200,200,1,2,3


In [29]:
df_check = pd.read_sql("SELECT * FROM api_fetch", engine)
display(df_check)

,id,travel_times_response,traffic_alerts_response,weather_alerts_response,tt_key,ta_key,wa_key,timestamp
0,1,200,200,200,1,2,3,2025-02-03 21:40:16


In [43]:
df_schema = pd.read_sql("DESCRIBE weather_alerts_raw", engine)
display(df_schema)

,Field,Type,Null,Key,Default,Extra
0,id,int unsigned,NO,PRI,None,
1,barometricpressure,float,YES,,None,
2,latitude,float,YES,,None,
3,longitude,float,YES,,None,
4,precipitationininches,float,YES,,None,
5,readingtime,datetime,YES,,None,
6,relativehumidity,float,YES,,None,
7,skycoverage,varchar(45),YES,,None,
8,stationid,varchar(45),YES,,None,
9,stationname,varchar(45),YES,,None,


In [44]:
cols = df_wa.columns
for col in cols:
    print(col)

BarometricPressure
Latitude
Longitude
PrecipitationInInches
ReadingTime
RelativeHumidity
SkyCoverage
StationID
StationName
TemperatureInFahrenheit
Visibility
WindDirection
WindDirectionCardinal
WindGustSpeedInMPH
WindSpeedInMPH
timestamp
wa_id
